<center><h1>**Use Cases**</h1></center>

In [3]:
#use case - 1 : This use case will illustrate creating a table in the database, inserting values into the table  and  joining 
#tables to query the data. 
#We have two tables - Users and Department. 
#The User table has the following fields:name, fullname , password and departments(foreign key)
#The Department table has the following fields:type of department,location of the department and users working there(foreign key)

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import sessionmaker

#Connecting to the SQL database
engine = create_engine('sqlite:///:memory:', echo=False)

#Creating a base class 
Base = declarative_base()

#Creating user class to be mapped to user table
class User(Base):
    __tablename__ = 'user'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password=Column(String)

def __repr__(self):
        return "<user(name='%s', fullname='%s',password='%s')>" % (self.name, self.fullname,self.password)

#Creating the user table into the database.
Base.metadata.create_all(engine)

#Creating objects to insert values into the user table
user_1=User(name="Anjana",fullname="Anjanah Chandrasekaran", password="anj1234")
user_2=User(name="Aiswrya",fullname="Aiswrya Bhaskaran", password="aish1234")
user_3=User(name="Junaid",fullname="Junaid Ali", password="Jun1234")
user_4=User(name="Monica",fullname="Monica L Perez", password="moni1234")

#The sessionmaker() creates a Session class. Whenever you want to "talk" to the database, you have to instantiate the class
Session = sessionmaker(bind=engine)

session = Session()
#using add to insert values into the table
session.add(user_1)
session.add(user_2)
session.add(user_3)
session.add(user_4)

#by using the commit() function, the changes in the database are finalized.
session.commit()

#using the query() function, we can select or update values into the table.
session.query(User).all()

#Sql queries
#Query 1: to select user name and password sort by user name
for instance in session.query(User).order_by(User.name):
    print(instance.name, instance.password)
     
#Query 2: to select all  user names
for row in session.query(User, User.name).all():
     print(row.User, row.name)

#Query 3: to count all  user names starting with A.
session.query(User).filter(User.name.like('A%')).count()

Aiswrya aish1234
Anjana anj1234
Junaid Jun1234
Monica moni1234
<__main__.User object at 0x00000231AC372898> Anjana
<__main__.User object at 0x00000231AC625DD8> Aiswrya
<__main__.User object at 0x00000231AC0AFCC0> Junaid
<__main__.User object at 0x00000231AC64E1D0> Monica


2

In [4]:
# Creating a new table department to illustrate join operaions
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

#Creating class department to be mapped to departments table
class departments(Base):
    __tablename__ = 'departments'
    id = Column(Integer, primary_key=True)
    loc=Column(String)
    Type=Column(String)
    user_id = Column(Integer, ForeignKey('user.id'))
    users = relationship("User", back_populates="departments")
#Defining the relationship between user table  and  departments table  
User.departments = relationship("departments", order_by = departments.id, back_populates = "users")

Base.metadata.create_all(engine)


In [6]:
#inserting values into the department table
user_1.departments = [departments(loc="Sanjose",Type="Sales")]
user_2.departments = [departments(loc="Seattle",Type="Operations")]
user_3.departments = [departments(loc="Manhattan",Type="Accounting")]
user_4.departments = [departments(loc="Chicago",Type="Marketing")]


#Query 4: This query joins two tables and displays the user name of those users whose department is in Chicago
session.query(User.name).join(departments).filter(departments.loc=="Chicago").all()




[('Monica')]

In [4]:
# Query 5: This query joins the two tables based on id and displays the departments each user belongs to  
for c, o in session.query(User, departments).filter(User.id == departments.user_id).all():
    print(c.fullname, o.Type)



Anjanah Chandrasekaran Sales
Aiswrya Bhaskaran Operations
Junaid Ali Accounting
Monica L Perez Marketing


In [5]:
#Use case 2 : This use case illustrates how a CSV file can be read into a SQL alchemy database and querying this data.
#We have a CSV file containing information about athlethes contesting in the Olympics, countries they are from, 
#the sports they excel at, the number of gold, silver and bronze medals won by them. 

import csv
import sqlalchemy as sa
import pandas as pd
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import sessionmaker

#Connecting to the SQL database
engine = sa.create_engine('sqlite:///:memory:', echo=True)
#creating a session to talk to the database
Session = sessionmaker(bind = engine)  

#read data from a csv file
lines = list(csv.reader(open('OlympicAthletes.csv',encoding = "ISO-8859-1")))
header = lines[0]
values = lines[1:]
#put the data into a pandas dataframe 
data = pd.DataFrame(data=values, columns = header)
#Write records stored in a DataFrame to a SQL database.
data.to_sql(name = 'Athletes', con = engine)

#create a class to handle the table Athletes
Base = declarative_base()
class Athletes(Base):
    __tablename__ = 'Athletes'
    index = Column(Integer, primary_key=True)
    Athlete = Column(String)
    Age = Column(Integer)
    Country = Column(String)
    Year = Column(String)
    ClosingCeremonyDate = Column("Closing Ceremony Date",String)
    Sport = Column(String)
    GoldMedals = Column("Gold Medals", Integer)
    SilverMedals = Column("Silver Medals", Integer)
    BronzeMedals = Column("Bronze Medals", Integer)
    TotalMedals = Column("Total Medals", Integer)

#Creating the schema of the Athletes table into the database.
Base.metadata.create_all(engine)

2017-02-18 18:03:10,774 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-02-18 18:03:10,778 INFO sqlalchemy.engine.base.Engine ()
2017-02-18 18:03:10,781 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-02-18 18:03:10,782 INFO sqlalchemy.engine.base.Engine ()
2017-02-18 18:03:10,785 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Athletes")
2017-02-18 18:03:10,787 INFO sqlalchemy.engine.base.Engine ()
2017-02-18 18:03:10,794 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Athletes" (
	"index" BIGINT, 
	"Athlete" TEXT, 
	"Age" TEXT, 
	"Country" TEXT, 
	"Year" TEXT, 
	"Closing Ceremony Date" TEXT, 
	"Sport" TEXT, 
	"Gold Medals" TEXT, 
	"Silver Medals" TEXT, 
	"Bronze Medals" TEXT, 
	"Total Medals" TEXT
)


2017-02-18 18:03:10,796 INFO sqlalchemy.engine.base.Engine ()
2017-02-18 18:03:10,799 INFO sqlalchemy.engine.base.Engine COMMIT
2017-02-18 18:03:10,801 INFO sqlalchemy.engine.

In [6]:
#creating a session to talk to the database
session = Session()
#Sql queries, importing func which allows to perform mathematical functions like sum(), min(),max()
from sqlalchemy import func
#1. Query that lists all the athletes from India
for instance in session.query(Athletes.Athlete).filter_by(Country = 'India').distinct(Athletes.Athlete):
    print(instance.Athlete)
    


2017-02-18 18:03:13,756 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-02-18 18:03:13,759 INFO sqlalchemy.engine.base.Engine SELECT DISTINCT "Athletes"."Athlete" AS "Athletes_Athlete" 
FROM "Athletes" 
WHERE "Athletes"."Country" = ?
2017-02-18 18:03:13,761 INFO sqlalchemy.engine.base.Engine ('India',)
Yogeshwar Dutt
Sushil Kumar
Karnam Malleswari
Vijay Kumar
Gagan Narang
Abhinav Bindra
Rajyavardhan Rathore
M. C. Mary Kom
Vijender Singh
Saina Nehwal


In [7]:
#2. Query lists the countries with highest number of medals
for c in session.query(Athletes.Country).group_by(Athletes.Country).order_by(func.sum(Athletes.TotalMedals).desc()): 
    print(c.Country)

2017-02-18 18:03:17,042 INFO sqlalchemy.engine.base.Engine SELECT "Athletes"."Country" AS "Athletes_Country" 
FROM "Athletes" GROUP BY "Athletes"."Country" ORDER BY sum("Athletes"."Total Medals") DESC
2017-02-18 18:03:17,044 INFO sqlalchemy.engine.base.Engine ()
United States
Russia
Germany
Australia
China
Canada
Italy
Great Britain
France
Netherlands
South Korea
Japan
Brazil
Spain
Norway
Cuba
Sweden
Hungary
Ukraine
Argentina
Romania
Finland
Belarus
Switzerland
Austria
Denmark
Croatia
Czech Republic
Jamaica
Poland
Greece
New Zealand
Kazakhstan
Bulgaria
Kenya
Nigeria
Mexico
Serbia and Montenegro
Slovakia
Serbia
Lithuania
Ethiopia
Turkey
Azerbaijan
Slovenia
South Africa
Bahamas
Iran
Georgia
Chile
Indonesia
North Korea
Cameroon
Chinese Taipei
Trinidad and Tobago
Uzbekistan
Belgium
Estonia
Thailand
Latvia
Paraguay
Iceland
Montenegro
Colombia
India
Morocco
Armenia
Mongolia
Ireland
Portugal
Algeria
Egypt
Singapore
Zimbabwe
Saudi Arabia
Dominican Republic
Moldova
Israel
Tunisia
Venezuela
Hong

In [8]:
#3. Query returns the Country and sport ordered by highest medal count.

for c in session.query(Athletes.Country,Athletes.Sport,func.sum(Athletes.TotalMedals).label('Total')).group_by(Athletes.Country,Athletes.Sport).order_by(func.sum(Athletes.TotalMedals).desc()): 
    print(c.Country,c.Sport,c.Total)

2017-02-18 18:03:17,649 INFO sqlalchemy.engine.base.Engine SELECT "Athletes"."Country" AS "Athletes_Country", "Athletes"."Sport" AS "Athletes_Sport", sum("Athletes"."Total Medals") AS "Total" 
FROM "Athletes" GROUP BY "Athletes"."Country", "Athletes"."Sport" ORDER BY sum("Athletes"."Total Medals") DESC
2017-02-18 18:03:17,651 INFO sqlalchemy.engine.base.Engine ()
United States Swimming 267
Australia Swimming 163
United States Athletics 147
Netherlands Hockey 113
United States Ice Hockey 106
Canada Ice Hockey 105
Russia Athletics 98
United States Basketball 95
Australia Hockey 81
Great Britain Rowing 80
Jamaica Athletics 80
Brazil Volleyball 72
Cuba Baseball 72
Brazil Football 70
Russia Volleyball 70
Germany Canoeing 68
Argentina Hockey 65
Germany Hockey 65
Finland Ice Hockey 64
Sweden Ice Hockey 64
United States Football 64
United States Waterpolo 63
Australia Rowing 61
United States Rowing 60
Great Britain Cycling 58
Italy Fencing 57
China Diving 55
Netherlands Rowing 55
United States

In [9]:
#4. Query returns the Maximum age of all athletes for each country

for c in session.query(Athletes.Country,func.max(Athletes.Age).label('MaxAGe')) .group_by(Athletes.Country):
     print(c.Country,c.MaxAGe)

2017-02-18 18:03:20,958 INFO sqlalchemy.engine.base.Engine SELECT "Athletes"."Country" AS "Athletes_Country", max("Athletes"."Age") AS "MaxAGe" 
FROM "Athletes" GROUP BY "Athletes"."Country"
2017-02-18 18:03:20,960 INFO sqlalchemy.engine.base.Engine ()
Afghanistan 25
Algeria 29
Argentina 46
Armenia 28
Australia 46
Austria 38
Azerbaijan 46
Bahamas 35
Bahrain 27
Barbados 24
Belarus 44
Belgium 42
Botswana 18
Brazil 47
Bulgaria 37
Cameroon 32
Canada 61
Chile 33
China 43
Chinese Taipei 31
Colombia 35
Costa Rica 27
Croatia 38
Cuba 38
Cyprus 22
Czech Republic 41
Denmark 46
Dominican Republic 34
Ecuador 34
Egypt 32
Eritrea 22
Estonia 42
Ethiopia 32
Finland 40
France 50
Gabon 23
Georgia 39
Germany 51
Great Britain 54
Greece 35
Grenada 19
Guatemala 21
Hong Kong 29
Hungary 39
Iceland 35
India 34
Indonesia 32
Iran 32
Ireland 32
Israel 28
Italy 51
Jamaica 40
Japan 41
Kazakhstan 38
Kenya 36
Kuwait 45
Kyrgyzstan 24
Latvia 33
Lithuania 36
Macedonia 26
Malaysia 29
Mauritius 30
Mexico 36
Moldova 33
Mong